# MatrixViewer Notebook

# Setup General Environment (always run this first)

In [ ]:
from timeit import default_timer as timer
import numpy as np
import pathlib
import math
import sys
import re
import pickle
import os
import warnings
from collections import OrderedDict
from IPython.core.display import display, HTML
warnings.simplefilter(action='ignore', category=FutureWarning)
display(HTML("<style>.container { width:100% !important; height:100% important}</style>"))
plot_width  = int(1200)
plot_height = int(plot_width//1.2)


####################################### Set data directories ##########################################
start_dir=os.getcwd() # Base directory (used for relative paths)
MTX_DATA_DIR = "MTX" # Input data directory containing matrix market matrices (used for plotting and AMGX solves)
PKL_DATA_DIR = "MTX_PKL" # Output data directory for pickled dataframes (used for plotting)
PETSC_DATA_DIR = "MTX_PETSC" # INPUT/OUTPUT directory for petsc matrices (used for petsc and slepc solves)



# Read Matrix market matrices and pickle data (For visualization)

In [ ]:
import pandas as pd
n_max_num_matrices_per_file = 2
n_max_lines = sys.maxsize

start = timer()
regex_start = r"%%MatrixMarket"                             # Start of matrix read:    %%MatrixMarket
regex_dims = r"([0-9]+)\s+([0-9]+)\s+([0-9]+)"              # Matrix dimensions:       int int int
regex_element = r"\s*([0-9]+)\s+([0-9]+)\s+([0-9\+\-\.e]+)" # Repeated Matrix entries: int int float
os.chdir(start_dir)
print(f"running in {os.getcwd()}")
max_row_id = -1
max_col_id = -1
min_val = sys.maxsize
max_val = -sys.maxsize
n_lower = 0
n_upper = 0
n_zero = 0
n_diag = 0
n_lines = 0
data_frames = OrderedDict()
file_list = {}

for file in os.listdir(MTX_DATA_DIR):
    if file.endswith(".mtx"):
        file_list[file] = open(os.path.join(MTX_DATA_DIR, file),'r')
    
if not os.path.isdir(PKL_DATA_DIR):
    os.mkdir(os.path.join(os.getcwd(), PKL_DATA_DIR))
os.chdir(PKL_DATA_DIR)

# Looad matrix market format matrix from one of more files (one per file)
for file, file_handle in file_list.items():
    orig_file_name = pathlib.Path(file).name
    file_name = re.sub("/","-",orig_file_name)
    data_name = ""
    mat_num = 0
    read_matrix = False
    get_matrix_dims = False
    with file_handle as file:
        for line in file:
            n_lines += 1
            if n_lines > n_max_lines:
                break
            # Check for start of matrix (%%MatrixMarket)
            if re.match(regex_start, line):
                read_matrix = True
                get_matrix_dims = True
                nans = 0
                n_data = 0
                d_s = {'row': [], 'col': [], 'val': [] }
                continue
            # Check for dimensions (int int int)
            if read_matrix and get_matrix_dims:
                dims = re.match(regex_dims, line)
                if dims:
                    dim1 = dims.group(1)
                    dim2 = dims.group(2)
                    dim3 = dims.group(3)
                    print(f"MATRIX{str(mat_num)} dimensions: {dim1} {dim2} {dim3}")
                    read_matrix = True
                    get_matrix_dims = False
                    continue
            # Check for element (int int float)
            if read_matrix:
                if re.search(regex_element, line):
                    matches = re.finditer(regex_element, line)
                    for matchNum, match in enumerate(matches):
                        for groupNum in range(1, len(match.groups()) + 1):
                            if groupNum == 1:
                                row = int(match.group(groupNum))
                            elif groupNum == 2:
                                col = int(match.group(groupNum))
                            elif groupNum == 3:
                                val = float(match.group(groupNum))
                    n_data += 1
                    d_s['row'].append(row)
                    d_s['col'].append(col)
                    d_s['val'].append(val)
                    if math.isnan(val):
                        nans += 1
                    max_row_id = max(max_row_id, row)
                    max_col_id = max(max_col_id, col)
                    min_val = min(min_val, val)
                    max_val = max(max_val, val) 
                    if abs(val) > 0.0:
                        if row > col:
                            n_lower += 1
                        elif col > row:
                            n_upper += 1
                        else:
                            n_diag += 1
                    else:
                        n_zero += 1
        # Dump pickle file
        filename = f"{orig_file_name}_MATRIX{str(mat_num)}_{dim1}_{dim2}_{dim3}"
        print(f"End of {filename} (records={n_data}, nans={nans})")
        data_frames[data_name] = pd.DataFrame(data=d_s)
        pickle_file_name = filename + ".p"
        f = open(pickle_file_name, 'wb')
        pickle.dump(data_frames[data_name], f, protocol=4)
        print("created pickle file " + pickle_file_name)
        f.close()
        mat_num += 1
        read_matrix = False
        get_matrix_dims = False

os.chdir(start_dir)
data_frames = OrderedDict()
for file in os.listdir(PKL_DATA_DIR):
    if file.endswith(".p"):
        file_name = PKL_DATA_DIR + "-" + re.split('\.',file)[0]
        f = open(os.path.join(PKL_DATA_DIR, file), 'rb')
        data_frames[file_name] = pickle.load(f, encoding='bytes')
        print("loaded " + file_name)
        f.close()

end = timer()
print("Data input complete: " + str(n_lines) + " lines. " + str(end - start) + " s")
print("diag, lower, upper, zero: " + ", ".join([str(n_diag), str(n_lower), str(n_upper), str(n_zero)]))
import io
print (io.DEFAULT_BUFFER_SIZE)


# Read pickle files (Fast data reload for visualization)

In [ ]:
# Load pickle files to speed up load of large matrices, for plotting
start = timer()
os.chdir(start_dir)
data_frames = OrderedDict()
for file in os.listdir(PKL_DATA_DIR):
    if file.endswith(".p"):
        file_name = PKL_DATA_DIR + "-" + re.split('\.',file)[0]
        f = open(os.path.join(PKL_DATA_DIR, file), 'rb')
        data_frames[file_name] = pickle.load(f, encoding='bytes')
        print("loaded " + file_name)
        f.close() 
end = timer()
print(str(end - start) + " s")

# Matrix plots 

In [ ]:
import holoviews as hv
import datashader as ds
import pandas as pd
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread, stack
from holoviews.streams import RangeXY
from datashader import transfer_functions as tf
from holoviews.operation import decimate
from datashader.colors import Sets1to3 # default datashade() and shade() color cycle
hv.extension('bokeh')
hv.notebook_extension('bokeh')
decimate.max_samples=1000
dynspread.max_px=20
dynspread.threshold=0.5

%opts RGB [width=plot_width, height=plot_height, invert_xaxis=False, invert_yaxis=True, xaxis='top'] {+axiswise}

scatter_dict = {}
color_key = [('positive', '#247ffe'), ('negative', '#e65036')]
colors = hv.NdOverlay({k: hv.Points([0,0], label=str(k)).opts(style=dict(color=v)) for k, v in color_key})
for data_name in data_frames:
    df = data_frames[data_name]
    plot = {'negative': hv.Scatter(df.loc[(df['val'] < 0.0)], kdims=['col', 'row']), 'positive': hv.Scatter(df.loc[(df['val'] > 0.0)], kdims=['col', 'row'])}
    plot_data = hv.NdOverlay(plot, kdims='sign')
    scatter_dict[data_name] = plot_data

hmap = dynspread(datashade(hv.HoloMap(scatter_dict, kdims=['data_name']), aggregator=ds.count_cat('sign')), threshold=0.75, how='over') * colors
hmap

# Matrix Aggregate Data (dynamic min, mean, max within window)

In [ ]:
import holoviews as hv
import datashader as ds
import pandas as pd
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread, stack
from holoviews.streams import RangeXY
import colorcet as cc
from datashader import transfer_functions as tf
from holoviews.operation import decimate
from datashader.colors import Sets1to3 # default datashade() and shade() color cycle
hv.extension('bokeh')
hv.notebook_extension('bokeh')
decimate.max_samples=1000
dynspread.max_px=20
dynspread.threshold=0.5

%opts QuadMesh [tools=['hover']] (alpha=0 hover_alpha=0.2)
%opts RGB [width=plot_width, height=plot_height, invert_xaxis=False, invert_yaxis=True, xaxis='top'] {+axiswise}

ccmap = list(reversed(cc.b_diverging_bwr_55_98_c37))

# Set aggregate type for window
aggregate_type = "mean" 
if aggregate_type == "mean": # Display mean value of data within window
    aggr = ds.mean('val')
elif aggregate_type == "min": # Display min value of data within window
    aggr = ds.min('val')
elif aggregate_type == "max": # Display max value of data within window
    aggr = ds.max('val')
elif aggregate_type == "variance": # Display variance of data within window
    aggr = ds.var('val')
elif aggregate_type == "count": # Display number of data elements within window
    aggr = ds.count('val')
elif aggregate_type == "sum": # Display num of data within window
    aggr = ds.sum('val')

# Set data id to plot
data_id = 0
value_filter = 0.0
data_names = []
for data_name in data_frames:
    print("data_id: " + str(len(data_names)) + ", data_name: " + data_name)
    df = data_frames[data_name]
    data_names.append(data_name)

df = data_frames[data_names[data_id]]
points = hv.Points(data=df.loc[(abs(df['val']) > value_filter)], kdims=['col','row'],vdims =['val'])
pts = dynspread(datashade(points, width=plot_width, height=plot_height, 
                         aggregator=ds.mean('val'),normalization='eq_hist',
                         cmap = ccmap), threshold=0.5, how='over')
(pts * hv.util.Dynamic(hd.aggregate(points, width=15, height=15, streams=[RangeXY],aggregator=aggr),
                               operation=hv.QuadMesh).relabel("Dynamic hover"))
 

# Save PETSC matrices

In [ ]:
import scipy.io
import petsc

# export PETSC_CONFIGURE_OPTIONS="--download-fblaslapack=1"

# Convert matrix market matrices to PETSc binary format
petsc_path = os.path.dirname(petsc.__file__) # Get petsc install location
os.environ['PETSC_DIR'] = petsc_path
sys.path.insert(0, petsc_path+'/lib/petsc/bin/pythonscripts/')
sys.path.insert(0, petsc_path+'/lib/petsc/bin/')

import PetscBinaryIO

print("PETSC_DIR=" + petsc_path)

file_list = {}
os.chdir(start_dir)
for file in os.listdir(MTX_DATA_DIR):
    if file.endswith(".mtx"):
        file_list[os.path.join(MTX_DATA_DIR, file)] = open(os.path.join(MTX_DATA_DIR, file),'r')
        
if not os.path.isdir(PETSC_DATA_DIR):
    os.mkdir(os.path.join(os.getcwd(), PETSC_DATA_DIR))
    
start = timer()
for file, file_handle in file_list.items():
    if file.endswith(".mtx"):
        print(file)
        output_file = re.split('\.',file)[0]
        print(output_file)
        A = scipy.io.mmread(file)
        mfile = open(re.sub(MTX_DATA_DIR, PETSC_DATA_DIR, output_file),'w')
        PetscBinaryIO.PetscBinaryIO().writeMatSciPy(mfile, A)
        print("converted " + file)
end = timer()
print(str(end - start) + " s")


# Solve with PETSC

In [ ]:
%matplotlib notebook
from matplotlib import pylab
import petsc4py
from petsc4py import PETSc
petsc4py.init(sys.argv)

# Matrix to solve
matrix_file = "1138_bus"

# Load petsc binary matrix
os.chdir(start_dir)
os.chdir(PETSC_DATA_DIR)
viewer = PETSc.Viewer().createBinary(matrix_file, 'r')
A = PETSc.Mat().load(viewer)
os.chdir(start_dir)

# Initialise Krylov solver
ksp = PETSc.KSP()
ksp.create(PETSc.COMM_WORLD)

# obtain sol & rhs vectors
x, b = A.createVecs()
x.set(0)
b.set(1)
ksp.setOperators(A)

# Solver parameters:

# use flexible gmres
ksp.setType('fgmres')

# use amg preconditioner
ksp.getPC().setType('gamg')
ksp.setConvergenceHistory(length=100, reset=False)
ksp.setComputeSingularValues(True)
r = ksp.getResidualNorm()
ksp.logConvergenceHistory(r)
ksp.setTolerances(rtol=0.000000001, max_it=100)
ksp.setComputeEigenvalues(True)

# Solve Ax = b
start = timer()
ksp.solve(b, x)
end = timer()

# Obtain eigenvalue data
eigen = ksp.computeEigenvalues()
(A, P) = ksp.getOperators()
n = ksp.getIterationNumber()
smin, smax = ksp.computeExtremeSingularValues()

# Solver output
print(f"extreme singular values: {str(smin)},{str(smax)}")
print(f"iterations: {str(n)}")
print(x[0:3])
print(f"solve time: {str(end - start)}")

# Plot solution, eigenvalues and residual reduction
fig, axs = pylab.subplots(3)
fig.tight_layout(pad=3.0)
#axs[0].hist(eigen.real, 50, normed=1, facecolor='green', alpha=0.75)
axs[0].scatter(eigen.real, eigen.imag, color='b')
axs[1].plot(x)
axs[2].plot(ksp.getConvergenceHistory()[1:])
axs[2].set_yscale("log")
axs[0].set_title("eigenvalues")
axs[1].set_title("solution")
axs[2].set_title("residual reduction")
pylab.show()

# Analyze eigenvalues with SLEPC

In [ ]:
%matplotlib notebook
from matplotlib import pylab
import slepc4py
import petsc4py
slepc4py.init(sys.argv)
from petsc4py import PETSc
from slepc4py import SLEPc

# Matrix to analyze
matrix_file = "1138_bus"

def print_results(title, E, axs, color):

    print()
    print(f"*** {title} ***")
    print()

    its = E.getIterationNumber()
    print("Number of iterations of the method: %d" % its)

    eps_type = E.getType()
    print("Solution method: %s" % eps_type)

    nev, ncv, mpd = E.getDimensions()
    print("Number of requested eigenvalues: %d" % nev)

    tol, maxit = E.getTolerances()
    print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))

    nconv = E.getConverged()
    print("Number of converged eigenpairs %d" % nconv)

    if nconv > 0:
        x = []
        y = []
        # Create the results vectors
        vr, wr = A.getVecs()
        vi, wi = A.getVecs()
        #
        print()
        print("        k          ||Ax-kx||/||kx|| ")
        print("----------------- ------------------")
        for i in range(nconv):
            k = E.getEigenpair(i, vr, vi)
            error = E.computeError(i)
            if i < 3:
                if k.imag != 0.0:
                    print(" %9f%+9f j %12g" % (k.real, k.imag, error))
                else:
                    print(" %12f      %12g" % (k.real, error))
            x.append(k.real)
            y.append(k.imag)
        print()
        axs.scatter(x, y, color=color)
        axs.set_title(title)

# set maximum dimension for direct solve
max_eigenpairs = 2000

# Load PETSc binary matrix
os.chdir(start_dir)
os.chdir(PETSC_DATA_DIR)
viewer = PETSc.Viewer().createBinary(matrix_file, 'r')
A = PETSc.Mat().load(viewer)
m = A.getSizes()
print(f"Matrix dimensions: {str(m)}")
os.chdir(start_dir)

# Initialize SLEPc solver
E = SLEPc.EPS()
E.create()
E.setOperators(A)
E.setProblemType(SLEPc.EPS.ProblemType.NHEP)

start = timer()
if m[0][0] > max_eigenpairs / 2:
    
    # Set plot parameters
    fig, axs = pylab.subplots(2)
    fig.tight_layout(pad=3.0)
    
    # KrylovSchur method for largest eigenvalues
    E.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
    E.setWhichEigenpairs(SLEPc.EPS.Which.LARGEST_MAGNITUDE)
    E.setDimensions(nev=(max_eigenpairs / 10))
    E.solve()
    print_results("KRYLOVSHCUR Largest", E, axs[0], 'r')
    
    # KylovSchur method with harmonic extraction for smallest eigenvalues
    E.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
    E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_MAGNITUDE)
    E.setExtraction(SLEPc.EPS.Extraction.HARMONIC)
    E.setDimensions(nev=(max_eigenpairs / 10))
    E.solve()
    print_results("KRYLOVSHCUR Smallest", E, axs[1], 'g')
else:
    
    # Direct solve using LAPACK
    fig, axs = pylab.subplots(1)
    fig.tight_layout(pad=3.0)
    E.setType(SLEPc.EPS.Type.LAPACK)
    E.solve()
    print_results("LAPACK Solve", E, axs[0], 'b')

end = timer()
print(f"Computation time: {str(end - start)}")

pylab.show()


# Solve with AMGX

In [ ]:
import scipy.sparse as sparse
import scipy.sparse.linalg as splinalg
import scipy.io
import pyamgx

# Matrix to solve
matrix_file = "1138_bus.mtx"

# Initialize amgx, unless already initialized
try:
    pyamgx.initialize()
except:
    print("failed to initialize amgx: attmpting to reinitialize")
    pyamgx.finalize()
    pyamgx.initialize()
finally:
    print("Initialized amgx")
    

# Initialize config and resources:
cfg_dict = {
    "config_version": 2, 
    "determinism_flag": 1,
    "exception_handling" : 1,
    "solver": {
        "print_grid_stats": 1, 
        "store_res_history": 1, 
        "solver": "GMRES", 
        "print_solve_stats": 1, 
        "obtain_timings": 1, 
        "preconditioner": {
            "interpolator": "D2", 
            "print_grid_stats": 1, 
            "solver": "AMG", 
            "smoother": "JACOBI_L1", 
            "presweeps": 2, 
            "selector": "PMIS", 
            "coarsest_sweeps": 2, 
            "coarse_solver": "NOSOLVER", 
            "max_iters": 1, 
            "interp_max_elements": 4, 
            "min_coarse_rows": 2, 
            "scope": "amg_solver", 
            "max_levels": 24, 
            "cycle": "V", 
            "postsweeps": 2
        }, 
        "max_iters": 100, 
        "monitor_residual": 1, 
        "gmres_n_restart": 10, 
        "convergence": "RELATIVE_INI_CORE", 
        "tolerance": 1e-06, 
        "norm": "L2"
   }
}
cfg = pyamgx.Config().create_from_dict(cfg_dict)
rsc = pyamgx.Resources().create_simple(cfg)

# Create matrices and vectors:
A = pyamgx.Matrix().create(rsc)
b = pyamgx.Vector().create(rsc)
x = pyamgx.Vector().create(rsc)

# Create solver:
solver = pyamgx.Solver().create(rsc, cfg)

# Load matrix market format matrix
os.chdir(start_dir)
os.chdir(MTX_DATA_DIR)
M = sparse.csr_matrix(scipy.io.mmread(matrix_file))
os.chdir(start_dir)

rhs = np.ones(M.shape[0])
sol = np.zeros(M.shape[0])
A.upload_CSR(M)
b.upload(rhs)
x.upload(sol)

# Setup
solver.setup(A)

# Solve system
start = timer()
solver.solve(b, x)
end = timer()

# Download solution
x.download(sol)
n = solver.iterations_number

print("pyamgx solution: ", sol)
print(f"iterations: {n}")
#print("scipy solution: ", splinalg.spsolve(M, rhs))
print(f"solve time: {str(end - start)}")

# Clean up:
A.destroy()
x.destroy()
b.destroy()
solver.destroy()
rsc.destroy()
cfg.destroy()

pyamgx.finalize()